In [46]:
import pandas as pd
import re
from pyquery import PyQuery as pq

In [47]:
# WARNING:  This is a horrible hack

lines = []
for e in range(1, 46):
    with open('offline/episode{:02}.htm'.format(e), 'r') as f:
        epi_doc = f.read()
    episode = pq(epi_doc)
    
    episode_name = episode('h1').text()
    
    segments = {}
    for link in episode('a[href*="#"]'):
        segments[link.attrib['href'][1:]] = link.text
        
    actors = {}
    for actor in episode[0].xpath('//center[contains(text(), "Colour code")]//font'):
        actors[actor.attrib['id']] = actor.text  
        
    segment = ''

    for row in episode("table")("tr").items():
        if row.attr("valign") == "top":
            character = row('td:first').text().strip().replace('<br>', ' ')
            text = str(row('td:eq(1)')).replace('\n', ' ').replace('<br>', ' ')
            text = re.search(r'<td>(.*?)</td>', text).group(1)
            splits = []
            for r in re.finditer(r'<[A-Za-z](.*?)>', text):
                t = '</{}>'.format(r.group(0)[:r.group(0).find(' ')][1:])
                end = text.find(t, r.start())
                if '<BR>' not in r.group():
                    splits.append({
                        'start': r.start(),
                        'end': end,
                        'stag': r.group(),
                        'etag': t
                    })
            bits = []            
            end = 0
            if len(splits) >0 and splits[0]['start'] > 1:
                bits.append(text[0:splits[0]['start']])
            for i in range(0, len(splits)):
                start = splits[i]['start']
                end = splits[i]['end']
                if end == -1:
                    end = 99999
                end += len(splits[i]['etag'])
                try:
                    next_tag = splits[i+1]['start']
                except IndexError:
                    next_tag = 99999
                if next_tag < end:
                    end = next_tag
                bits.append(text[start:end])
            bits.append(text[end:])
            current_tag = ''
            current_closer = ''
            new_bits = []
            for bit in bits:
                tag = re.search(r'<[A-Za-z](.*?)>', bit)
                if tag:
                    if tag.group()[0:2] == '<a':
                        new_bits.append(tag.group())
                        bit = current_tag + bit.replace(tag.group(),'')
                    t = '</{}>'.format(tag.group(0)[:tag.group(0).find(' ')][1:])
                    closer = bit.find(t)
                    if closer < 0:
                        current_tag = tag.group(0)
                        current_closer = '</{}>'.format(tag.group(0)[:tag.group(0).find(' ')][1:])
                        bit = bit + t
                else:
                    bit = current_tag + bit + current_closer
                if len(bit.strip()) > 0:
                    new_bits.append(bit)
            for nb in new_bits:
                try:
                    doc_temp = pq(nb)                    
                    action = nb[0:2].upper()
                    if action == '<A':
                        segment = segments[doc_temp('a').attr('name')]
                        actor = ''
                        detail = ''
                        stype = ''
                    elif action == '<I':
                        stype = 'Direction'
                        actor = ''
                        detail = doc_temp.text()
                    elif action == '<F':
                        stype = 'Dialogue'
                        actor = actors[doc_temp('font').attr('id')]
                        detail = doc_temp.text()
                    elif action == '<P':
                        stype = 'Direction'
                        actor = ''
                        detail = doc_temp('p').text()
                    else:
                        actor = ''
                        detail = doc_temp.text()
                    act = re.search(r'\((.*?)\)', character)
                    if act:
                        use_character = character.replace(act.group(), '')
                        actor = act.group()[1:-1]
                        if action != '<I':
                            stype = 'Dialogue'
                    else:
                        use_character = character
                    if action != '<A':
                        if len(detail.strip()) > 0:
                            if detail.strip() != '(':
                                if detail[0] == ')':
                                    detail = detail[1:]
                                if detail[0] == '(' and detail[-1] == ')':
                                    detail = detail[1:-1]
                                lines.append({
                                    'episode': e,
                                    'episode_name': episode_name.strip(),
                                    'segment': segment.strip(),
                                    'character': use_character.strip(),
                                    'actor': actor.strip(),
                                    'type': stype.strip(),
                                    'detail': detail.strip(),
                                }) 
                except:
                    pass
                
df = pd.DataFrame(lines)
df = df[['episode', 'episode_name', 'segment', 'type', 'actor', 'character', 'detail']].drop_duplicates()
df.to_excel('excel/full.xlsx'.format(e))